# Задание из четвёртой лабы

Обучите один из алгоритмов классификации из библиотеки sklearn. Объяснять принцип работы классификатора в этой лабораторной не требуется

1. Выберите один из датасетов первой лабораторной


2. Найдите в данных признак, который будете предсказывать
    
    Если это категория, то у неё должно быть от 2 до 10 значений (например колонка species в датасете миграции птиц содержит 3 категории)
    
    Если признак количественный, то его нужно дискретизировать (например для популярности трека в датасете spotify можно разделить треки на 3 категории: 0-20 - популярные, 21-50 - обычные, 51 - 100 - непопулярные). Полученную категорию записать в датафрейм как отдельную колонку


3. Выделите признаки для предсказания
    
    Используйте от 2 до 5 количественных признаков для предсказания целевой переменной


4. Проверьте нормальность количественных признаков
    
    Используйте статистический тест из лекции. Если распределение является лог-нормальным, логарифмируйте его и проверьте на нормальность
    
    Как минимум 1 из признаков должен быть нормальным


5. Покажите качество предсказания модели с помощью метрик

    Метрика должна учитывать баланс классов в датасете

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, recall_score, precision_score, f1_score
import pandas as pd
from copy import copy
import numpy as np
import statistics as st
from scipy import stats

In [2]:
ds = pd.read_csv('content/SpotifyFeatures.csv')
idx = ds[ds.isna().any(axis=1)].index
ds = ds.drop(index=idx)
ds.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [3]:
ds.loc[ds['mode'] == 'Major', 'mode'] = 0
ds.loc[ds['mode'] == 'Minor', 'mode'] = 1
ds.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,0,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,1,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,1,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,0,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,0,0.0456,140.576,4/4,0.390


In [4]:
ds_size = len(ds)
split_ratio = 0.8
split_index = int(split_ratio * ds_size)

ds['normal_column'] = np.random.normal(size=len(ds))

ds.loc[ds['popularity'] < 20, 'popularity_category'] = 1
ds.loc[ds['popularity'] >= 20, 'popularity_category'] = 2
ds.loc[ds['popularity'] >= 40, 'popularity_category'] = 3

X = copy(ds).drop(['mode', 'track_id', 'track_name', 
                   'artist_name', 'genre', 'key', 'time_signature'], axis=1).values.tolist()

y = ds['mode'].values.tolist()

X_train, X_val = X[split_index:], X[:split_index]
y_train, y_val = y[split_index:], y[:split_index]

In [37]:
_, counts = np.unique(y, return_counts=True)
counts

array([151744,  80981])

In [38]:
forest = RandomForestClassifier(n_estimators=10, criterion="entropy")
forest.fit(X_train, y_train)
forest.score(X_val, y_val)

0.655795466752605

In [40]:
y_pred = forest.predict(X_val)

precision = precision_score(y_val, y_pred) # tp / (tp + fp)
recall = recall_score(y_val, y_pred) # tp / (tp + fn)
f1 = f1_score(y_val, y_pred)

print(f'Precision: {precision}, Recall: {recall}, F1: {f1}')

Precision: 0.5318922406839813, Recall: 0.2495572274110292, F1: 0.33972139795581935


In [47]:
stats.normaltest(ds['normal_column'])

NormaltestResult(statistic=0.6743510888354204, pvalue=0.7137835281903289)